In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn

In [2]:
import urllib
import json
url_link = 'http://elections.huffingtonpost.com/pollster/api/charts/?topic=2014-senate'

response = urllib.request.urlopen(url_link).read()
info = json.loads(response)
type(info)

list

In [3]:
from collections import Counter
Countstates = Counter([entry['state'] for entry in info])
twopoll = [s for s in Countstates if Countstates[s]>1]
twopoll

['OK', 'SC', 'NH']

In [6]:
anomaly = {i: entry['title'] for i, entry in enumerate(info) if entry['state'] in twopoll}
anomaly

{4: '2014 Oklahoma Senate: Lankford vs. Johnson',
 12: '2014 Oklahoma Senate: Inhofe vs. Silverstein',
 13: '2014 South Carolina Senate: Scott vs. Dickerson',
 14: '2014 South Carolina Senate: Graham vs. Hutto',
 29: '2014 New Hampshire Senate: Bass vs. Shaheen',
 31: '2014 New Hampshire Senate: Brown vs. Shaheen'}

In [7]:
rem = set([29, 4, 14])
info = [entry for i, entry in enumerate(info) if i not in rem]

In [10]:
polls = {entry['title']: pd.read_csv(entry['url']+'.csv') for entry in info}

In [11]:
info[0]

{'id': 571,
 'title': '2014 Delaware Senate: Wade vs. Coons',
 'slug': '2014-delaware-senate-wade-vs-coons',
 'topic': '2014-senate',
 'state': 'DE',
 'short_title': '2014 Delaware Senate: Wade vs. Coons',
 'election_date': '2014-11-04',
 'poll_count': 5,
 'last_updated': '2016-06-06T14:37:54.000Z',
 'url': 'https://elections.huffingtonpost.com/pollster/2014-delaware-senate-wade-vs-coons',
 'estimates': [{'choice': 'Coons',
   'value': 56.0,
   'lead_confidence': 99.9,
   'first_name': 'Chris',
   'last_name': 'Coons',
   'party': 'Dem',
   'incumbent': True},
  {'choice': 'Wade',
   'value': 34.0,
   'lead_confidence': 0.0,
   'first_name': 'Kevin',
   'last_name': 'Wade',
   'party': 'Rep',
   'incumbent': False}]}

In [36]:
estimates = [entry['estimates'] for entry in info]
def candidateof(party, cred = 'party'):
    L = []
    for est in estimates:
        for can in est:
            if can[cred] == party:
                L.append(can['first_name'] + ' ' + can['last_name'])
                break
        else:
            L.append(None)
    return L
def incum():
    L = []
    for est in estimates:
        for can in est:
            if can['incumbent'] == True:
                L.append(can['party'][0])
                break
        else:
            L.append(None)
    return L

In [37]:
candidates = pd.DataFrame()
candidates['State'] = [entry['state'] for entry in info]
candidates['R'] = candidateof('Rep')
candidates['D'] = candidateof('Dem')
candidates['incumbent'] = incum()

In [38]:
candidates

,State,R,D,incumbent
0,DE,Kevin Wade,Chris Coons,D
1,MA,Brian Herr,Ed Markey,D
2,RI,None,None,None
3,MT,Steve Daines,Amanda Curtis,None
4,HI,Campbell Cavasso,Brian Schatz,D
5,KS,Pat Roberts,None,R
6,WY,Mike Enzi,Charlie Hardy,R
7,TN,Lamar Alexander,Gordon Ball,R
8,ID,Jim Risch,Nels Mitchell,R
9,NJ,Jeff Bell,Cory Booker,D


In [40]:
polls['2014 Delaware Senate: Wade vs. Coons']

,Pollster,Start Date,End Date,Entry Date/Time (ET),Number of Observations,Population,Mode,Coons,Wade,Undecided,Other,Pollster URL,Source URL,Partisan,Affiliation,Question Text,Question Iteration
0,CBS/NYT/YouGov,2014-10-16,2014-10-23,2014-10-26T14:52:36Z,461,Likely Voters,Internet,54.0,36.0,10.0,0.0,https://elections.huffingtonpost.com/pollster/...,https://today.yougov.com/news/2014/09/07/battl...,Nonpartisan,None,NaN,1
1,CBS/NYT/YouGov,2014-09-20,2014-10-01,2014-10-05T15:35:39Z,471,Likely Voters,Internet,51.0,35.0,14.0,0.0,https://elections.huffingtonpost.com/pollster/...,https://today.yougov.com/news/2014/09/07/battl...,Nonpartisan,None,NaN,1
2,University of Delaware,2014-09-10,2014-09-22,2014-09-26T19:49:46Z,769,Registered Voters,Live Phone,50.0,25.0,11.0,16.0,https://elections.huffingtonpost.com/pollster/...,http://www.udel.edu/cpc/research/fall2014/CPCN...,Nonpartisan,None,NaN,1
3,Rasmussen,2014-09-10,2014-09-12,2014-09-15T15:30:24Z,750,Likely Voters,IVR/Online,49.0,34.0,12.0,4.0,https://elections.huffingtonpost.com/pollster/...,http://www.rasmussenreports.com/public_content...,Nonpartisan,None,NaN,1
4,CBS/NYT/YouGov,2014-08-18,2014-09-02,2014-09-07T15:51:31Z,697,Likely Voters,Internet,48.0,35.0,16.0,1.0,https://elections.huffingtonpost.com/pollster/...,https://today.yougov.com/news/2014/09/07/battl...,Nonpartisan,None,NaN,1
